In [1]:
import numpy as NP

from astroutils import nonmathops as NMO

from ClosureInvariants import graphUtils as GU
from ClosureInvariants import scalarInvariants as SI

from IPython.core.debugger import set_trace

## Set up antennas and array info

In [2]:
element_ids = ['O', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
nelements_total = len(element_ids)
element_locs = 1e3*NP.random.randn(nelements_total,2) # in lambda units
elements_subset = ['O', 'A', 'C', 'E', 'G']
elements_remove = ['B', 'D', 'F']
nelements_subset = len(elements_subset)
element_indices_subset = NMO.find_list_in_list(element_ids, elements_subset)
element_locs_subset = element_locs[element_indices_subset,:]
print(element_indices_subset)

[0 1 3 5 7]


## Determine antenna pairs

In [3]:
element_pairs = [(element_ids[i], element_ids[j]) for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]
npairs = len(element_pairs)
element_pairs_locs = NP.array([element_locs[i] - element_locs[j] for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]) # in lambda units
element_pairs_subset = [(elements_subset[i], elements_subset[j]) for i in range(len(elements_subset)) for j in range(i+1,len(elements_subset))]
npairs_subset = len(element_pairs_subset)
element_pairs_indices = [(i,j) for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]
element_pairs_subset_indices = [(element_indices_subset[i],element_indices_subset[j]) for i in range(len(elements_subset)) for j in range(i+1,len(elements_subset))]
indices_of_subset_pairs = NMO.find_list_in_list(list(map(str,element_pairs)), list(map(str,element_pairs_subset))) # Indices of subset element pairs in all element pairs
element_pairs_subset_locs = element_pairs_locs[indices_of_subset_pairs]
print(element_pairs)
print(element_pairs_indices)
print(element_pairs_subset)
print(element_pairs_subset_indices)
print(indices_of_subset_pairs)

[('O', 'A'), ('O', 'B'), ('O', 'C'), ('O', 'D'), ('O', 'E'), ('O', 'F'), ('O', 'G'), ('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('A', 'F'), ('A', 'G'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('B', 'F'), ('B', 'G'), ('C', 'D'), ('C', 'E'), ('C', 'F'), ('C', 'G'), ('D', 'E'), ('D', 'F'), ('D', 'G'), ('E', 'F'), ('E', 'G'), ('F', 'G')]
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]
[('O', 'A'), ('O', 'C'), ('O', 'E'), ('O', 'G'), ('A', 'C'), ('A', 'E'), ('A', 'G'), ('C', 'E'), ('C', 'G'), ('E', 'G')]
[(0, 1), (0, 3), (0, 5), (0, 7), (1, 3), (1, 5), (1, 7), (3, 5), (3, 7), (5, 7)]
[0 2 4 6 8 10 12 19 21 26]


## Determine the complete and independent set of triads

In [4]:
baseid = 'O'
base_ind = element_ids.index(baseid)
base_ind_subset = elements_subset.index(baseid)
triads_indep = GU.generate_independent_triads(element_ids, baseid='O')
triads_subset_indep = GU.generate_independent_triads(elements_subset, baseid='O')
indices_subset_triads = NMO.find_list_in_list(list(map(str,triads_indep)), list(map(str,triads_subset_indep))) # Indices of subset triads in all triads
print(triads_indep)
print(triads_subset_indep)
print(indices_subset_triads)

[('O', 'A', 'B'), ('O', 'A', 'C'), ('O', 'A', 'D'), ('O', 'A', 'E'), ('O', 'A', 'F'), ('O', 'A', 'G'), ('O', 'B', 'C'), ('O', 'B', 'D'), ('O', 'B', 'E'), ('O', 'B', 'F'), ('O', 'B', 'G'), ('O', 'C', 'D'), ('O', 'C', 'E'), ('O', 'C', 'F'), ('O', 'C', 'G'), ('O', 'D', 'E'), ('O', 'D', 'F'), ('O', 'D', 'G'), ('O', 'E', 'F'), ('O', 'E', 'G'), ('O', 'F', 'G')]
[('O', 'A', 'C'), ('O', 'A', 'E'), ('O', 'A', 'G'), ('O', 'C', 'E'), ('O', 'C', 'G'), ('O', 'E', 'G')]
[1 3 5 12 14 19]


## Generate mock cross-correlations

In [5]:
nruns_shape = (16,12,10) # Shape of number of runs (for example, number of random realisations, timestamps, channels)
npol = 1
bl_axis = -1 # Baseline axis
xc_real_std = 1.0
xc_imag_stad = 1.0
randseed = None
rng = NP.random.default_rng(randseed)
xcorr = rng.normal(loc=0.0, scale=xc_real_std, size=nruns_shape+(npairs,)) + 1j * rng.normal(loc=0.0, scale=xc_real_std, size=nruns_shape+(npairs,)) # The last axis is the baseline axis
xcorr_subset = NP.take(xcorr, indices_of_subset_pairs, axis=bl_axis) # Take a subset of the baseline axes based on the elements subset
print(xcorr.shape)
print(xcorr_subset.shape)

(16, 12, 10, 28)
(16, 12, 10, 10)


## Set up triangular loop correlations

In [6]:
corrs_lol = SI.corrs_list_on_loops(xcorr, element_pairs, triads_indep, bl_axis=bl_axis) # _lol stands for list of lists
corrs_lol_subset_method1 = SI.corrs_list_on_loops(xcorr, element_pairs, triads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
corrs_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(corrs_lol))
print(len(corrs_lol_subset_method1))
print(len(corrs_lol_subset_method2))
print(NP.max(NP.abs(NP.array(corrs_lol_subset_method1)-NP.array(corrs_lol_subset_method2)))) # Verify that both methods of setup produce identical results

21
6
6
0.0


## Compute advariants

In [7]:
advariants = SI.advariants_multiple_loops(corrs_lol)
advariants_subset = SI.advariants_multiple_loops(corrs_lol_subset_method1)
print(advariants.shape)
print(advariants_subset.shape)

(16, 12, 10, 21)
(16, 12, 10, 6)


## Compute the invariants by removing the unknown common scaling factor

In [8]:
cloinv = SI.invariants_from_advariants_method1(advariants, normaxis=-1, normwts=None, normpower=2)
cloinv_subset = SI.invariants_from_advariants_method1(advariants_subset, normaxis=-1, normwts=None, normpower=2)
print(cloinv.shape)
print(cloinv_subset.shape)

(16, 12, 10, 42)
(16, 12, 10, 12)


## Test the invariance of the closure invariants obtained by corrupting the correlations using element-based gains

## Generate mock copolar gains

In [9]:
element_axis = -1 # Antenna axis
mean_gain_scale = 3.0 
element_gains = rng.normal(loc=1.0, scale=NP.sqrt(0.5)/mean_gain_scale, size=nruns_shape+(nelements_total,)).astype(NP.float64) + 1j * rng.normal(loc=1.0, scale=NP.sqrt(0.5)/mean_gain_scale, size=nruns_shape+(nelements_total,)).astype(NP.float64) # shape is (...,n_element)
element_gains_subset = NP.take(element_gains, element_indices_subset, axis=element_axis)
print(element_gains.shape)
print(element_gains_subset.shape)

(16, 12, 10, 8)
(16, 12, 10, 5)


## Corrupt the correlations

In [10]:
prefactor_gains = NP.take(element_gains, NP.array(element_pairs_indices)[:,0], axis=element_axis) # A collection of g_a
postfactor_gains = NP.take(element_gains, NP.array(element_pairs_indices)[:,1], axis=element_axis) # A collection of g_b
xcorr_mod = SI.corrupt_visibilities(xcorr, prefactor_gains, postfactor_gains)

prefactor_gains_subset = NP.take(element_gains, NP.array(element_pairs_subset_indices)[:,0], axis=element_axis) # A collection of g_a
postfactor_gains_subset = NP.take(element_gains, NP.array(element_pairs_subset_indices)[:,1], axis=element_axis) # A collection of g_b
xcorr_subset_mod = SI.corrupt_visibilities(xcorr_subset, prefactor_gains_subset, postfactor_gains_subset)

print(xcorr_mod.shape)
print(xcorr_subset_mod.shape)

(16, 12, 10, 28)
(16, 12, 10, 10)


## Compute the closure invariants through advariants, and scale factor elimination

In [11]:
corrs_mod_lol = SI.corrs_list_on_loops(xcorr_mod, element_pairs, triads_indep, bl_axis=bl_axis)
corrs_mod_lol_subset_method1 = SI.corrs_list_on_loops(xcorr_mod, element_pairs, triads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
corrs_mod_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset_mod, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(corrs_mod_lol))
print(len(corrs_mod_lol_subset_method1))
print(len(corrs_mod_lol_subset_method2))
print(NP.max(NP.abs(NP.array(corrs_mod_lol_subset_method1)-NP.array(corrs_mod_lol_subset_method2)))) # Verify that both methods of setup produce identical results

21
6
6
0.0


In [12]:
advariants_mod = SI.advariants_multiple_loops(corrs_mod_lol)
advariants_mod_subset = SI.advariants_multiple_loops(corrs_mod_lol_subset_method1)
print(advariants_mod.shape)
print(advariants_mod_subset.shape)

(16, 12, 10, 21)
(16, 12, 10, 6)


In [13]:
cloinv_mod = SI.invariants_from_advariants_method1(advariants_mod, normaxis=-1, normwts=None, normpower=2)
cloinv_mod_subset = SI.invariants_from_advariants_method1(advariants_mod_subset, normaxis=-1, normwts=None, normpower=2)
print(cloinv_mod.shape)
print(cloinv_mod_subset.shape)

(16, 12, 10, 42)
(16, 12, 10, 12)


## Check for invariance between ideal and modified versions

In [14]:
print(NP.allclose(cloinv, cloinv_mod))
print(NP.allclose(cloinv_subset, cloinv_mod_subset))

True
True


## Verify scale factors are as expected, namely absolute squared value of the copolar complex gain of the base vertex

In [15]:
scale_factor = NP.abs(element_gains)**2
scale_factor_subset = NP.abs(element_gains_subset)**2

In [16]:
print(NP.allclose(scale_factor[...,[base_ind]], advariants_mod / advariants))
print(NP.allclose(scale_factor_subset[...,[base_ind_subset]], advariants_mod_subset / advariants_subset))

True
True


## Point source verification

### Generate random point-like object location, intensity, and visibilities

In [17]:
src_loc_complex = NP.random.uniform() * NP.exp(1j*NP.random.uniform(low=-NP.pi, high=NP.pi))
src_loc = NP.array([src_loc_complex.real, src_loc_complex.imag]).reshape(1,-1) # in (l,m) coordinates
src_loc_0 = NP.zeros((1,2), dtype=float) # at origin / phase centre
src_amp = NP.random.rand() # in Jy
src_vis = NP.sum(src_amp * NP.exp(2j*NP.pi * NP.dot(element_pairs_locs, src_loc.T)), axis=-1)
src_vis_subset = src_vis[indices_of_subset_pairs]
src_0_vis = NP.sum(src_amp * NP.exp(2j*NP.pi * NP.dot(element_pairs_locs, src_loc_0.T)), axis=-1)
src_0_vis_subset = src_0_vis[indices_of_subset_pairs]

In [18]:
print(src_loc)
print(src_loc_0)
print(src_vis.shape)
print(src_vis_subset.shape)

[[ 0.46663699 -0.45153329]]
[[0. 0.]]
(28,)
(10,)


### Generate advariants for the source visibilities

In [19]:
src_corrs_lol = SI.corrs_list_on_loops(src_vis, element_pairs, triads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_vis_subset, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_advariants = SI.advariants_multiple_loops(src_corrs_lol)
src_advariants_subset = SI.advariants_multiple_loops(src_corrs_lol_subset_method2)

src_0_corrs_lol = SI.corrs_list_on_loops(src_0_vis, element_pairs, triads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_0_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_0_vis_subset, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_0_advariants = SI.advariants_multiple_loops(src_0_corrs_lol)
src_0_advariants_subset = SI.advariants_multiple_loops(src_0_corrs_lol_subset_method2)


In [20]:
print(src_advariants.shape)

(21,)


In [21]:
src_cloinv_normpow2 = SI.invariants_from_advariants_method1(src_advariants, normaxis=-1, normwts=None, normpower=2)
src_cloinv_subset_normpow2 = SI.invariants_from_advariants_method1(src_advariants_subset, normaxis=-1, normwts=None, normpower=2)
src_cloinv_normpow1 = SI.invariants_from_advariants_method1(src_advariants, normaxis=-1, normwts=None, normpower=1)
src_cloinv_subset_normpow1 = SI.invariants_from_advariants_method1(src_advariants_subset, normaxis=-1, normwts=None, normpower=1)

src_0_cloinv_normpow2 = SI.invariants_from_advariants_method1(src_0_advariants, normaxis=-1, normwts=None, normpower=2)
src_0_cloinv_subset_normpow2 = SI.invariants_from_advariants_method1(src_0_advariants_subset, normaxis=-1, normwts=None, normpower=2)
src_0_cloinv_normpow1 = SI.invariants_from_advariants_method1(src_0_advariants, normaxis=-1, normwts=None, normpower=1)
src_0_cloinv_subset_normpow1 = SI.invariants_from_advariants_method1(src_0_advariants_subset, normaxis=-1, normwts=None, normpower=1)

### Verify invariance to source translation

In [22]:
print(NP.allclose(src_cloinv_normpow2, src_0_cloinv_normpow2))
print(NP.allclose(src_cloinv_normpow1, src_0_cloinv_normpow1))

True
True


### Verify point source closure invariants with "max" normalisation of advariants

In [23]:
src_cloinv_maxnorm_normpow1 = SI.invariants_from_advariants_method1(src_advariants, normaxis=-1, normwts='max', normpower=1)
src_cloinv_subset_maxnorm_normpow1 = SI.invariants_from_advariants_method1(src_advariants_subset, normaxis=-1, normwts='max', normpower=1)

In [24]:
expected_src_advariant = NP.ones_like(src_advariants)
expected_src_cloinv_maxnorm_normpow1 = NP.concatenate([expected_src_advariant.real, expected_src_advariant.imag], axis=-1)

expected_src_advariant_subset = NP.ones_like(src_advariants_subset)
expected_src_cloinv_subset_maxnorm_normpow1 = NP.concatenate([expected_src_advariant_subset.real, expected_src_advariant_subset.imag], axis=-1)

In [25]:
print(NP.allclose(src_cloinv_maxnorm_normpow1, expected_src_cloinv_maxnorm_normpow1))
print(NP.allclose(src_cloinv_subset_maxnorm_normpow1, expected_src_cloinv_subset_maxnorm_normpow1))

True
True


### Verify point source closure phases

In [26]:
src_cphases = SI.closurePhases_from_advariants(src_advariants)
print(NP.allclose(src_cphases, NP.zeros(src_advariants.shape)))

True


## Generate independent quads

In [27]:
quads_indep = GU.generate_independent_quads(element_ids)
quads_subset_indep = GU.generate_independent_quads(elements_subset)
print(len(quads_subset_indep))
print(quads_subset_indep)
print('----------')
print(len(quads_indep))
print(quads_indep)

5
[('A', 'C', 'E', 'G'), ('A', 'C', 'G', 'O'), ('C', 'E', 'G', 'O'), ('O', 'A', 'C', 'E'), ('O', 'A', 'E', 'G')]
----------
20
[('A', 'B', 'C', 'D'), ('A', 'B', 'D', 'E'), ('A', 'B', 'E', 'F'), ('A', 'B', 'F', 'G'), ('A', 'B', 'G', 'O'), ('B', 'C', 'D', 'E'), ('B', 'C', 'E', 'F'), ('B', 'C', 'F', 'G'), ('B', 'C', 'G', 'O'), ('C', 'D', 'E', 'F'), ('C', 'D', 'F', 'G'), ('C', 'D', 'G', 'O'), ('D', 'E', 'F', 'G'), ('D', 'E', 'G', 'O'), ('E', 'F', 'G', 'O'), ('O', 'A', 'B', 'C'), ('O', 'A', 'C', 'D'), ('O', 'A', 'D', 'E'), ('O', 'A', 'E', 'F'), ('O', 'A', 'F', 'G')]


## Set up quadrilateral loop correlations

In [28]:
quad_corrs_lol = SI.corrs_list_on_loops(xcorr, element_pairs, quads_indep, bl_axis=bl_axis) # _lol stands for list of lists
quad_corrs_lol_subset_method1 = SI.corrs_list_on_loops(xcorr, element_pairs, quads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
quad_corrs_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(quad_corrs_lol))
print(len(quad_corrs_lol_subset_method1))
print(len(corrs_lol_subset_method2))
print(NP.max(NP.abs(NP.array(quad_corrs_lol_subset_method1)-NP.array(quad_corrs_lol_subset_method2)))) # Verify that both methods of setup produce identical results

20
5
6
0.0


## Compute Covariants

In [29]:
covariants = SI.covariants_multiple_loops(quad_corrs_lol)
covariants_subset = SI.covariants_multiple_loops(quad_corrs_lol_subset_method1)
print(covariants.shape)
print(covariants_subset.shape)

(16, 12, 10, 20)
(16, 12, 10, 5)


## Compute Closure Amplitudes from Covariants

In [30]:
clAmp = SI.closureAmplitudes_from_covariants(covariants)
clAmp_subset = SI.closureAmplitudes_from_covariants(covariants_subset)
print(clAmp.shape)
print(clAmp_subset.shape)

(16, 12, 10, 20)
(16, 12, 10, 5)


## Compute closure amplitudes to gain corruptions

In [31]:
quad_corrs_mod_lol = SI.corrs_list_on_loops(xcorr_mod, element_pairs, quads_indep, bl_axis=bl_axis)
quad_corrs_mod_lol_subset_method1 = SI.corrs_list_on_loops(xcorr_mod, element_pairs, quads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
quad_corrs_mod_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset_mod, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(quad_corrs_mod_lol))
print(len(quad_corrs_mod_lol_subset_method1))
print(len(quad_corrs_mod_lol_subset_method2))
print(NP.max(NP.abs(NP.array(quad_corrs_mod_lol_subset_method1)-NP.array(quad_corrs_mod_lol_subset_method2)))) # Verify that both methods of setup produce identical results

20
5
5
0.0


In [32]:
covariants_mod = SI.covariants_multiple_loops(quad_corrs_mod_lol)
covariants_mod_subset = SI.covariants_multiple_loops(quad_corrs_mod_lol_subset_method1)
print(covariants_mod.shape)
print(covariants_mod_subset.shape)

(16, 12, 10, 20)
(16, 12, 10, 5)


In [33]:
clAmp_mod = SI.closureAmplitudes_from_covariants(covariants_mod)
clAmp_mod_subset = SI.closureAmplitudes_from_covariants(covariants_mod_subset)
print(clAmp_mod.shape)
print(clAmp_mod_subset.shape)

(16, 12, 10, 20)
(16, 12, 10, 5)


## Check for invariance between ideal and modified versions

In [34]:
print(NP.allclose(clAmp, clAmp_mod))
print(NP.allclose(clAmp_subset, clAmp_mod_subset))

True
True


## Point source verification

### Generate covariants for the source visibilities

In [35]:
src_quad_corrs_lol = SI.corrs_list_on_loops(src_vis, element_pairs, quads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_quad_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_vis_subset, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_covariants = SI.covariants_multiple_loops(src_quad_corrs_lol)
src_covariants_subset = SI.covariants_multiple_loops(src_quad_corrs_lol_subset_method2)

src_0_quad_corrs_lol = SI.corrs_list_on_loops(src_0_vis, element_pairs, quads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_0_quad_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_0_vis_subset, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_0_covariants = SI.covariants_multiple_loops(src_0_quad_corrs_lol)
src_0_covariants_subset = SI.covariants_multiple_loops(src_0_quad_corrs_lol_subset_method2)

### Verify invariance of covariants to source translation. If that is satisfied, closure amplitudes, which are the amplitudes of covariants will also be satisfied.

In [36]:
print(NP.allclose(src_covariants, src_0_covariants))
print(NP.allclose(src_covariants_subset, src_0_covariants_subset))

True
True


### Verify invariance of closure amplitudes to source translation

In [37]:
src_clAmp = SI.closureAmplitudes_from_covariants(src_covariants)
src_0_clAmp = SI.closureAmplitudes_from_covariants(src_0_covariants)
print(NP.allclose(src_clAmp, src_0_clAmp))

True


### Verify point source closure amplitudes

In [38]:
print(NP.allclose(src_clAmp, NP.ones(src_covariants.shape)))

True
